<h1 style="text-align: center; color: #E30613;"><b><i>Entraînement avec DziriBert sur les Données de Sentiments</i></b></h1>

<p style="font-size: 18px;">
Ce notebook présente un workflow complet pour l'entraînement et l'évaluation d'un modèle de classification des sentiments en utilisant <span style="color: #28A745;"><b>DziriBert</b></span>, un modèle de langage pré-entraîné.
</p>

### <span style="color: #28A745;">**Objectifs :**</span>
1. Charger et prétraiter les données textuelles.
2. Encoder les étiquettes des sentiments.
3. Utiliser un tokenizer pour préparer les données pour le modèle.
4. Diviser les données en ensembles d'entraînement, de validation et de test.
5. Entraîner le modèle avec des techniques avancées comme le dropout et l'early stopping.
6. Évaluer les performances du modèle à l'aide de métriques comme la matrice de confusion et le rapport de classification.
7. Visualiser les courbes de pertes et d'accuracy pour analyser les performances.

### <span style="color: #28A745;">**Plan du Notebook :**</span>
1. **Introduction et Bibliothèques nécessaires**  
    Importation des bibliothèques et configuration de l'environnement.
    
2. **Chargement et Prétraitement des Données**  
    Nettoyage des données et préparation des étiquettes.

3. **Tokenization et Préparation des Données**  
    Utilisation du tokenizer DziriBert et création des ensembles d'entraînement, de validation et de test.

4. **Entraînement du Modèle**  
    Entraînement avec des techniques comme l'early stopping et le dropout.

5. **Évaluation et Visualisation**  
    Analyse des performances avec des métriques et des visualisations.

6. **Conclusion**  
    Résumé des résultats et perspectives d'amélioration.

# <span style="color: #E30613;">**DziriBert**</span>

## <span style="color: #28A745;">**Bibiliothèques nécessaires**</span>

In [1]:
import torch
from torch.utils.data import DataLoader, Dataset, random_split
from torch.nn import functional as F
from torch import nn
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig, get_scheduler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import numpy as np
import seaborn as sns
import random

## <span style="color: #28A745;">**Utilisation de GPU**</span>

In [2]:
# ✅ Configuration du device GPU uniquement
assert torch.cuda.is_available(), "CUDA GPU is not available."
device = torch.device("cuda")

## <span style="color: #28A745;">**Chargement des Données**</span>

In [3]:
# 📁 Chargement des données
df = pd.read_csv("/content/Results/Comments_clean.csv").dropna(subset=["Comments", "Sentiments"])

## <span style="color: #28A745;">**Encodage des étiquettes**</span>

In [4]:
# 🎯 Encodage des étiquettes
label_encoder = LabelEncoder()
df["Sentiments_encoded"] = label_encoder.fit_transform(df["Sentiments"])
num_labels = len(label_encoder.classes_)

## <span style="color: #28A745;">**Tokenizer, Split dataset (Train 70%, Test 20%, Val 10%)**</span>

In [5]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
from nltk.corpus import wordnet
import random
from transformers import AutoTokenizer
import torch
from torch.utils.data import Dataset, DataLoader, random_split

# Charger le tokenizer
tokenizer = AutoTokenizer.from_pretrained("alger-ia/dziribert")


import random
from nltk.corpus import wordnet

def get_synonyms(word):
    """ Trouve les synonymes d'un mot en utilisant WordNet. """
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name())
    if word in synonyms:
        synonyms.remove(word)
    return list(synonyms)

def augment_text(text, p_swap=0.2, p_del=0.1, p_syn=0.15):
    """ Applique diverses augmentations sur le texte. """
    words = text.split()
    if len(words) < 2: return text

    # 🔄 1️⃣ Remplacement par synonymes
    if random.random() < p_syn:
        for i, word in enumerate(words):
            synonyms = get_synonyms(word)
            if synonyms and random.random() > 0.5:
                words[i] = random.choice(synonyms)

    # 🔀 2️⃣ Permutation de mots adjacents
    if random.random() < p_swap:
        idx = list(range(len(words) - 1))
        random.shuffle(idx)
        for i in idx[:max(1, len(idx) // 5)]:
            words[i], words[i + 1] = words[i + 1], words[i]

    # ❌ 3️⃣ Suppression aléatoire de mots (maximum 2 mots)
    if random.random() < p_del and len(words) > 2:
        idx_to_remove = random.sample(range(len(words)), k=min(2, len(words) // 4))
        words = [word for i, word in enumerate(words) if i not in idx_to_remove]

    return " ".join(words)


# 🎯 **Dataset avec augmentation**
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128, augment=False):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.augment = augment

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])

        # ➡️ Appliquer l'augmentation seulement pendant l'entraînement
        if self.augment:
            text = augment_text(text)

        encoding = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(),
            "attention_mask": encoding["attention_mask"].squeeze(),
            "labels": torch.tensor(self.labels[idx], dtype=torch.long)
        }

# ✂️ Split dataset (Train 70%, Test 20%, Val 10%)
dataset = SentimentDataset(df["Comments"].values, df["Sentiments_encoded"].values, tokenizer, augment=True)
train_size = int(0.7 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_set, val_set, test_set = random_split(dataset, [train_size, val_size, test_size])

# ➡️ Charger les données
train_loader = DataLoader(train_set, batch_size=16, shuffle=True)
val_loader = DataLoader(val_set, batch_size=16)
test_loader = DataLoader(test_set, batch_size=16)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


## <span style="color: #28A745;">**Chargement du modèle avec dropout custom**</span>

In [7]:
"""
# Chargement du modèle avec dropout custom
config = AutoConfig.from_pretrained("alger-ia/dziribert", num_labels=num_labels, hidden_dropout_prob=0.45)
model = AutoModelForSequenceClassification.from_pretrained("alger-ia/dziribert", config=config)
model.to(device)
"""

config = AutoConfig.from_pretrained("alger-ia/dziribert", num_labels=num_labels, hidden_dropout_prob=0.4, output_attentions=True)
model = AutoModelForSequenceClassification.from_pretrained("alger-ia/dziribert", config=config)
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at alger-ia/dziribert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.4, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

## <span style="color: #28A745;">**Optimiseur et Scheduler**</span>

In [8]:
# Optimiseur et Scheduler
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)
num_epochs = 10
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler(
    "cosine",
    optimizer=optimizer,
    num_warmup_steps=100,
    num_training_steps=num_training_steps
)

## <span style="color: #28A745;">**Entraînement de Modèle**</span>

In [9]:
criterion = torch.nn.CrossEntropyLoss()

def mixup_data(x, y, alpha=0.1):
    """ Mélange les données et les labels avec un coefficient aléatoire. """
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(x.device)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

In [ ]:
# 🔧 Entraînement avec Early Stopping
best_val_loss = float("inf")
patience, patience_counter = 2, 0
train_losses, val_losses = [], []
train_accs, val_accs = [], []

# Entraînement avec Mixup et Dropout
for epoch in range(num_epochs):
    print(f"\n🟢 Epoch {epoch+1}/{num_epochs}")
    model.train()
    total_loss, correct, total = 0, 0, 0

    for batch in tqdm(train_loader, desc="Training"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # 💥 Mixup - Appliquer uniquement pendant l'entraînement
        if random.random() < 0.5:
            mixed_input_ids, labels_a, labels_b, lam = mixup_data(input_ids.float(), labels)
            mixed_input_ids = mixed_input_ids.long()  # Corriger le type après mixup
            outputs = model(input_ids=mixed_input_ids, attention_mask=attention_mask)
            loss = lam * criterion(outputs.logits, labels_a) + (1 - lam) * criterion(outputs.logits, labels_b)
        else:
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

        logits = outputs.logits

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        total_loss += loss.item()
        preds = torch.argmax(logits, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_loss = total_loss / len(train_loader)
    train_acc = correct / total
    train_losses.append(train_loss)
    train_accs.append(train_acc)

    # 🔍 Validation (sans Mixup)
    model.eval()
    val_loss, correct, total = 0, 0, 0
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validation"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

            preds = torch.argmax(outputs.logits, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    val_loss = val_loss / len(val_loader)
    val_acc = correct / total
    val_losses.append(val_loss)
    val_accs.append(val_acc)

    print(f"✅ Train Loss: {train_loss:.4f} | Acc: {train_acc:.4f} - Val Loss: {val_loss:.4f} | Acc: {val_acc:.4f}")

    # 🛑 Early Stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "/content/Results/best_model_DziriBert.pt")
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("⛔ Early stopping triggered.")
            break


🟢 Epoch 1/10


Training:  47%|████▋     | 75/159 [00:30<00:31,  2.71it/s]

## <span style="color: #28A745;">**Évaluation finale**</span>

In [ ]:
# Évaluation finale
model.load_state_dict(torch.load("/content/Results/best_model_DziriBert.pt"))
model.eval()
all_preds, all_labels = [], []

for batch in test_loader:
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    labels = batch["labels"].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    preds = torch.argmax(outputs.logits, dim=1)
    all_preds.extend(preds.cpu().numpy())
    all_labels.extend(labels.cpu().numpy())

print("🔍 Rapport de classification pour DziriBert:")
print(classification_report(all_labels, all_preds, target_names=label_encoder.classes_))

## <span style="color: #28A745;">**Courbes des pertes**</span>

In [ ]:
# Courbes des pertes
plt.plot(train_losses, label="Train Loss", color='#28A745')  # Vert
plt.plot(val_losses, label="Validation Loss", color='#E30613')  # Rouge
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training and Validation Loss")

plt.legend()
plt.show()

## <span style="color: #28A745;">**Matrice de confusion**</span>

In [ ]:
# Matrice de confusion
cm = confusion_matrix(all_labels, all_preds)
sns.heatmap(cm, annot=True, fmt='d', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_, cmap="Reds")
plt.xlabel("Prédictions")
plt.ylabel("Vérités")
plt.title("Matrice de confusion - DziriBert")
plt.show()

<h3 style="text-align: center; color: #E30613;"><b><i>Développé par: OUARAS Khelil Rafik</i></b></h3>